## Testing CUR IAM Role Permissions

### Positive Test Cases

1. **Get SageMaker Domain ID**
   - Should return the correct domain ID for a valid domain name.

2. **Get Account ID**
   - Should return the correct account ID of the current user.

3. **Validate IAM Role**
   - Should retrieve the role details, attached policies, and inline policies for a valid role name.

4. **SageMaker Permissions**
   - Should be able to list training jobs.
   - Should be able to describe a non-existent training job (it should fail gracefully indicating the job does not exist).

5. **Athena Permissions**
   - Should be able to list query executions.
   - Should be able to start a query execution.

6. **QuickSight Permissions**
   - Should be able to list dashboards for a valid account ID.

7. **S3 Permissions**
   - Should be able to list objects in a specified S3 bucket.

8. **CUR Permissions**
   - Should be able to describe report definitions.

### Negative Test Cases

1. **SageMaker Canvas Permissions**
   - Should fail to create, describe, or delete a Canvas app as the user does not have the necessary permissions.

In [ ]:
import boto3
from botocore.exceptions import ClientError

def get_sagemaker_domain_id(domain_name):
    sagemaker_client = boto3.client('sagemaker')
    try:
        response = sagemaker_client.list_domains()
        for domain in response['Domains']:
            if domain['DomainName'] == domain_name:
                return domain['DomainId']
        print(f"Domain {domain_name} not found.")
        return None
    except ClientError as e:
        print(f"Error retrieving SageMaker domain ID: {e}")
        return None

def get_account_id():
    sts_client = boto3.client('sts')
    try:
        response = sts_client.get_caller_identity()
        account_id = response['Account']
        return account_id
    except Exception as e:
        print(f"Error retrieving account ID: {e}")
        return None

def check_permissions(service_client, action, **kwargs):
    try:
        method = getattr(service_client, action)
        response = method(**kwargs)
        print(f" '{action}' succeeded")
        print(f"{response}")
        return True
    except ClientError as e:
        print(f" '{action}' failed")
        print(f"{e}")
        return False

def check_canvas_permissions(service_client, action, **kwargs):
    try:
        method = getattr(service_client, action)
        response = method(**kwargs)
        print(f"{action} incorrectly succeeded with response:")
        print(f"{response}")
        return True
    except ClientError as e:
        print(f"{action} failed successfully with error:")
        print(f"{e}")
        return False

def test_sagemaker_permissions():
    sagemaker_client = boto3.client('sagemaker')
    print("\n Testing SageMaker ListTrainingJobs permission")
    check_permissions(sagemaker_client, 'list_training_jobs')

def test_athena_permissions():
    athena_client = boto3.client('athena')
    print("\nTesting Athena ListQueryExecutions permission")
    check_permissions(athena_client, 'list_query_executions')

def test_quicksight_permissions(account_id):
    quicksight_client = boto3.client('quicksight')
    print("\nTesting QuickSight ListDashboards permission")
    check_permissions(quicksight_client, 'list_dashboards', AwsAccountId=account_id)

def test_cur_permissions():
    cur_client = boto3.client('cur')
    print("\nTesting CUR DescribeReportDefinitions permission")
    check_permissions(cur_client, 'describe_report_definitions')

def test_sagemaker_canvas_permissions(domain_id, user_profile_name):
    sagemaker_client = boto3.client('sagemaker')
    canvas_actions = [
        'create_app',
        'delete_app',
        'describe_app',
    ]
    for action in canvas_actions:
        print(f"\nTesting SageMaker Canvas {action} permission")
        if action == 'create_app':
            check_canvas_permissions(sagemaker_client, action, DomainId=domain_id, UserProfileName=user_profile_name, AppType='Canvas', AppName='TestCanvasApp')
        elif action == 'delete_app':
            check_canvas_permissions(sagemaker_client, action, DomainId=domain_id, UserProfileName=user_profile_name, AppType='Canvas', AppName='TestCanvasApp')
        elif action == 'describe_app':
            check_canvas_permissions(sagemaker_client, action, DomainId=domain_id, UserProfileName=user_profile_name, AppType='Canvas', AppName='TestCanvasApp')
        elif action == 'start_app':
            check_canvas_permissions(sagemaker_client, action, DomainId=domain_id, UserProfileName=user_profile_name, AppType='Canvas', AppName='TestCanvasApp')
        elif action == 'stop_app':
            check_canvas_permissions(sagemaker_client, action, DomainId=domain_id, UserProfileName=user_profile_name, AppType='Canvas', AppName='TestCanvasApp')

def test_permissions(account_id, domain_id, user_profile_name):
    test_sagemaker_permissions()
    print("")
    test_athena_permissions()
    print("")
    test_quicksight_permissions(account_id)
    print("")
    test_cur_permissions()
    print("")
    test_sagemaker_canvas_permissions(domain_id, user_profile_name)

role_name = 'SageMakerCostUsageReportsRole'
domain_name = 'SagemakerTestDomain'

account_id = get_account_id()
domain_id = get_sagemaker_domain_id(domain_name)

test_permissions(account_id, domain_id, role_name)
